### Exercício 1 - Projeto 2 - Lógica Computacional

Grupo G20

Nome: Lara Catarina Vilaça Lopes ; Número de aluno:a108655

Nome: Fábio Mendes Castelhano ; Número de aluno:a105728

1. Abordagem utilizada para resolver o problema

Utilização das bibliotecas z3, numpy, functools

In [2]:
from z3 import *
import numpy as np
from functools import reduce

Calcular o XOR lógico de uma lista de variáveis booleanas

In [ ]:
def xor_lista(lista):
    if not lista:
        return BoolVal(False)
    elif len(lista) == 1:
        return lista[0]
    else:
        # O lambda a, b: Xor(a, b) define uma função anônima que aplica Xor entre dois elementos, e o reduce faz isso acumulativamente ao longo da lista.
        return reduce(lambda a, b: Xor(a, b), lista)

- A função `gerar_vetores` tem (n, s) como input, onde:
  - n define o número de bits dos vetores a, b e c
  - s é a *seed* usada para gerar os valores aleatórios de forma reprodutível


- Produz usando um PRNG com s e gera-se os vetores $$\,a,b,c\in\{0,1\}^n\,$$ 

In [4]:
def gerar_vetores(s,n):
    rng = np.random.default_rng(seed=s)
    a = rng.integers(0, 2, size=n, dtype=np.uint8)
    b = rng.integers(0, 2, size=n, dtype=np.uint8)
    c = rng.integers(0, 2, size=n, dtype=np.uint8)
    return a, b, c


- A função `valor_polinomio` calcula o valor de um polinómio binário de 2º grau com base nos parâmetros: p = (o, a, b, c) onde:
    - $o \in \{0,1\}$ é o termo constante (offset)
    - $a, b, c \in \{0,1\}^n$ são vetores binários que selecionam subconjuntos de bits de entrada
    - $x \in \{0,1\}^n$ é o vetor de entrada

- O polinómio é definido como:
$$
f(p; x) \;\equiv\; o \;\oplus\; \langle a \cdot x \rangle \;\oplus\; \big(\langle b \cdot x \rangle \times \langle c \cdot x \rangle\big)
$$
onde:

- $\langle a \cdot x \rangle$ representa o XOR de todos os bits de $x$ selecionados por $a$
- $\langle b \cdot x \rangle \cdot \langle c \cdot x \rangle$ representa o produto lógico (AND) entre os XORs dos bits selecionados por $b$ e $c$


In [ ]:
def valor_polinomio(p, x):
    o, a, b, c = p
    ax = np.bitwise_xor.reduce(x[a == 1]) if np.any(a == 1) else 0
    bx = np.bitwise_xor.reduce(x[b == 1]) if np.any(b == 1) else 0
    cx = np.bitwise_xor.reduce(x[c == 1]) if np.any(c == 1) else 0
    return o ^ (ax ^ (bx & cx))
"""
queremos calcular o valor numérico do polinômio

Calcula o valor ax da seguinte forma:

a == 1 cria uma máscara booleana, selecionando apenas os elementos de x onde a tem 1.

x[a == 1] pega os elementos correspondentes de x.

np.bitwise_xor.reduce(...) aplica o XOR acumulativo em todos esses elementos.
→ Exemplo: se os elementos forem [1, 0, 1], o resultado é 1 ^ 0 ^ 1 = 0.

Se a não tiver nenhum 1 (np.any(a == 1) é falso), então ax = 0.

📘 Em resumo: ax é o XOR de todos os bits de x cujos índices correspondem a 1 em a.
x = [1, 0, 1, 1]  # vetor de valores
a = [0, 1, 1, 0]  # vetor “máscara”
a == 1 → [False, True, True, False]
posição 0 → máscara False → ignorado → valor x[0] = 1 não entra

posição 1 → máscara True → incluído → valor x[1] = 0 ✅

posição 2 → máscara True → incluído → valor x[2] = 1 ✅

posição 3 → máscara False → ignorado → valor x[3] = 1 não entra
x[a == 1] → [0, 1]

Se o número de bits 1 em x[a == 1] for ímpar, então ax = 1.

Se o número de bits 1 em x[a == 1] for par (ou zero), então ax = 0.


x = [1, 0, 1, 1]
a = [0, 1, 1, 0]

x[a == 1] = [0, 1]     # pegamos as posições onde a tem 1
0 ^ 1 = 1

"""

- A função `construir_SMT` constrói um modelo lógico SMT (Satisfiability Modulo Theories) que representa um conjunto de subcircuitos booleanos definidos pelos parâmetros fornecidos.  
Cada subcircuito é descrito por um polinómio binário de 2º grau da forma:

$$
f_i(p_i, x) = o_i \oplus \langle a_i, x \rangle \oplus (\langle b_i, x \rangle \land \langle c_i, x \rangle)
$$

onde:

- $x = (x_0, x_1, \dots, x_{n-1})$ é o vetor de bits de entrada  
- $o_i \in \{0,1\}$ é o termo constante (offset)  
- $a_i, b_i, c_i \in \{0,1\}^n$ são vetores binários que selecionam subconjuntos de bits de entrada  
- $\langle a_i, x \rangle$ representa o XOR de todos os bits $x_j$ tais que $a_{ij} = 1$, isto é:

  $$
  \langle a_i, x \rangle = \oplus_{j : a_{ij} = 1} x_j
  $$

- o produto lógico $\langle b_i, x \rangle \land \langle c_i, x \rangle$ representa o termo quadrático:

  $$
  \langle b_i, x \rangle \land \langle c_i, x \rangle
  $$

Em forma expandida, o polinómio pode ser escrito como:

$$
f_i(p_i, x) = o_i \oplus \left( \oplus_{j : a_{ij}=1} x_j \right) \oplus 
\left[ \left( \oplus_{j : b_{ij}=1} x_j \right) \land \left( \oplus_{j : c_{ij}=1} x_j \right) \right]
$$


In [ ]:
def construir_SMT(parametros, n, s_observado=None):
    solver = Solver()
    # três listas de variáveis booleanas simbólicas no Z3
    """
    x₀, x₁, ..., xₙ₋₁ → entrada.

    d₀, d₁, ..., dₙ₋₁ → variáveis de controle (defeitos, erros, etc.).

    s₀, s₁, ..., sₙ₋₁ → saídas esperadas.
    """
    x = [Bool(f"x_{i}") for i in range(n)]  
    d = [Bool(f"d_{i}") for i in range(n)]  
    y = [Bool(f"s_{i}") for i in range(n)] 

    for i in range(n):
        o, a, b, s = parametros[i]

        ax = xor_lista([x[j] for j in range(n) if a[j] == 1])
        bx = xor_lista([x[j] for j in range(n) if b[j] == 1])
        sx = xor_lista([x[j] for j in range(n) if s[j] == 1])

        and_ideal = And(bx, sx)
        """ 
        Ou seja, d[i] funciona como um “sinal de defeito”:
        quando ativo, ele força o termo AND a zero.
        """
        
        and_val = If(d[i], BoolVal(False), and_ideal)
        # Se houver falha (d[i] == True), força o resultado do AND a False; caso contrário, mantém o valor normal (and_ideal)
        # Sem falha → and_val = And(bx, sx)
        # Com falha → and_val = False
        pol_bool = Xor(BoolVal(bool(o)), ax, and_val)

        solver.add(y[i] == pol_bool)
        # A saída y[i] deve ser igual ao valor calculado do polinômio para este subcircuito

        if s_observado is not None:
            solver.add(pol_bool == BoolVal(bool(s_observado[i])))

    return solver, x, d, y


A função `gerar_parametros_subcircuitos` cria os parâmetros $(o_i, a_i, b_i, c_i)$ que definem cada subcircuito de 2º grau do circuito completo.

O seu objetivo é **gerar $n$ subcircuitos independentes**, cada um com vetores aleatórios $a_i, b_i, c_i$ e um termo constante $o_i$ calculado a partir de $z$.


In [ ]:
def gerar_parametros_subcircuitos(z, s, n):
    """ 
    🔹 Cria um gerador de números aleatórios mestre (rng_master) usando o NumPy e a semente s.
    Esse gerador vai ser usado para criar várias “sub-sementes”, garantindo que cada subcircuito tenha seus próprios vetores aleatórios
    """
    rng_master = np.random.default_rng(seed=s)
    parametros = []

    # Assim, cada subcircuito terá sua própria semente derivada da semente principal s.
    seeds = rng_master.integers(0, 2**32, size=n, dtype=np.uint32)

    for i in range(n):
        seed_i = seeds[i]
        a, b, c = gerar_vetores(seed_i, n)
        # Cria uma tupla p_i que representa um “polinômio” sem termo constante inicial (o = 0)
        #p_i -> polinomio base
        p_i = (0, a, b, c)
        # o resultado dessa avaliação (o_i) será o bit que falta para ajustar o circuito de forma consistente com z
        o_i = valor_polinomio(p_i, z)
        # guarda o offset obtido
        parametros.append((o_i, a, b, c))

    return parametros

A função `estimar_z_com_falhas` tenta resolver o modelo SMT construído anteriormente para estimar o vetor $z'$ e identificar possíveis falhas nos subcircuitos.

In [ ]:
def estimar_z_com_falhas(solver, x_vars, d_vars):
    if solver.check() == sat:
        modelo = solver.model()
        # Cria uma lista chamada z_estimado, com os valores concretos (True/False) que o solver atribuiu a cada variável x_i.
        """ 
        modelo.evaluate(xi) avalia o valor de x_i dentro do modelo.
        model_completion=True garante que, se alguma variável não foi explicitamente usada em restrições, o solver mesmo assim atribua a ela um valor padrão (False).
        O bool(...) converte o resultado simbólico do Z3 (True ou False) para um valor booleano Python normal.
        """
        z_estimado = [bool(modelo.evaluate(xi, model_completion=True)) for xi in x_vars]
        falhas = [bool(modelo.evaluate(di, model_completion=True)) for di in d_vars]
        return z_estimado, falhas
    else:
        print(" O modelo SMT é insatisfatível — não foi possível encontrar z' com falhas.")
        return None, None
# z_estimado é a melhor estimativa de z (entrada) que explica as saídas observadas e as restrições.

A função `maximizar_falhas_and` tenta encontrar a **configuração de falhas** $d = (d_0, d_1, \dots, d_{n-1})$ que **maximiza o número de "AND gates" falhadas**, respeitando uma limitação de tolerância $\varepsilon$, **sem alterar o resultado esperado do circuito** (isto é, mantendo $y_i = 0$).



- Parâmetros:

  - $z \in \{0,1\}^n$ — vetor conhecido de entrada (segredo real)  
  - parametros — lista de tuplos $(o_i, a_i, b_i, c_i)$, que definem os subcircuitos  
  - $n$ — dimensão do circuito (número de subcircuitos)  
  - $\varepsilon$ — probabilidade máxima de falha tolerada  



- Objetivo matemático

Maximizar o número de falhas nas *AND gates* de modo que o circuito continue a produzir a saída $0^n$:

$$
\begin{aligned}
\text{Maximize:} \quad & \sum_{i=0}^{n-1} d_i \\
\text{sujeito a:} \quad 
& f_i(p_i, z, d_i) = 0, \quad \forall i = 0, \dots, n-1 \\
& \sum_{i=0}^{n-1} d_i \leq \varepsilon \cdot n
\end{aligned}
$$

onde cada subcircuito é definido por:

$$
f_i(p_i, z, d_i) = o_i \oplus \langle a_i, z \rangle \oplus 
\left( 
  \langle b_i, z \rangle \land \langle c_i, z \rangle 
\right)_{\text{com falha}}
$$

e a falha na *AND gate* é modelada como:

$$
\left( \langle b_i, z \rangle \land \langle c_i, z \rangle \right)_{\text{com falha}} =
\begin{cases}
\text{False}, & \text{se } d_i = \text{True} \\
\langle b_i, z \rangle \land \langle c_i, z \rangle, & \text{caso contrário}
\end{cases}
$$

In [ ]:
# Encontrar a distribuição de falhas d que maximiza o número de falhas, sem ultrapassar o limite imposto por epsilon.
def maximizar_falhas_and(z, parametros, n, epsilon):
    opt = Optimize()
    x_vals = [BoolVal(bool(z[i])) for i in range(n)]
    """ 
     Cria uma lista de variáveis booleanas simbólicas (d_0, d_1, …, d_{n-1}) — cada uma representando se há falha ativa em um subcircuito (True = falha).
     """
    d = [Bool(f"d_{i}") for i in range(n)]

    falhas_totais = [] # 🔹 Inicializa uma lista para guardar expressões inteiras que representarão o número total de falhas ativas.
    for i in range(n): # 🔹 Percorre cada subcircuito para montar as restrições de cada um.
        o, a, b, c = parametros[i]
        ax = xor_lista([x_vals[j] for j in range(n) if a[j] == 1])
        bx = xor_lista([x_vals[j] for j in range(n) if b[j] == 1])
        cx = xor_lista([x_vals[j] for j in range(n) if c[j] == 1])
        # Esses valores representam as partes lógicas internas do subcircuito
        and_ideal = And(bx, cx) # 🔹 Calcula o AND ideal entre bx e cx (isto é, sem falhas).
        and_val = If(d[i], BoolVal(False), and_ideal) # 🔹 Introduz o efeito da falha:
        # Se d[i] é verdadeiro (falha presente) → força o valor do AND a False;
        #Caso contrário → mantém o valor ideal (And(bx, cx)).
        #Essa é a modelagem simbólica de um defeito que anula o AND.
        yi = Xor(ax, Xor(BoolVal(bool(o)), and_val)) # 🔹 Define a saída lógica do subcircuito:
        # onde and_val já considera o possível defeito.
        opt.add(yi == BoolVal(False)) # 🔹 Adiciona uma restrição que força o subcircuito i a produzir saída falsa (False).

        falhas_totais.append(If(d[i], 1, 0)) # 🔹 Adiciona um termo numérico que vale 1 se d[i] for verdadeiro (falha presente), senão 0 para somar

    max_falhas = int(epsilon * n) # 🔹 Define o número máximo permitido de falhas, calculando ε × n.
    opt.add(Sum(falhas_totais) <= max_falhas)

    opt.maximize(Sum(falhas_totais))

    if opt.check() == sat:
        modelo = opt.model()
        """ 
        🔹 Extrai os valores concretos de cada variável d_i (True se falha, False se não).
        is_true() é uma função auxiliar do Z3 para converter expressões simbólicas em booleanos Python.
        """
        falhas = [is_true(modelo[d[i]]) for i in range(n)]
        return falhas
    else:
        print("Não foi possível maximizar as falhas dentro do limite ε.")
        return None
"""
quanto maior o epsilon, maior o número máximo de falhas que o solver pode ativar.
epsilon funciona como um limite de tolerância, e o solver procura atingir o máximo permitido.
"""

2. Exemplos

In [15]:
n = 8
k = 32
epsilon = 0.3

z = np.random.randint(0, 2, size=n, dtype=np.uint8)
s = np.random.randint(0, 2**k, dtype=np.uint32)

parametros = gerar_parametros_subcircuitos(z, s, n)
solver, x, d, y = construir_SMT(parametros, n)
z_estimado, falhas = estimar_z_com_falhas(solver, x, d)
falhas_max = maximizar_falhas_and(z, parametros, n, epsilon)

print("z verdadeiro:     ", z.tolist())
print("z' estimado:      ", z_estimado)
print("Falhas estimadas: ", falhas)
print("Falhas maximizadas:", falhas_max)

print("Número de falhas toleradas:", sum(falhas_max) if falhas_max else 0)

z verdadeiro:      [0, 1, 0, 0, 1, 1, 1, 1]
z' estimado:       [True, False, True, True, False, False, True, False]
Falhas estimadas:  [False, False, False, False, False, False, False, False]
Falhas maximizadas: [False, False, False, False, False, True, False, True]
Número de falhas toleradas: 2


In [17]:
n = 8
k = 32
epsilon = 0.25

z = np.random.randint(0, 2, size=n, dtype=np.uint8)
s = np.random.randint(0, 2**k, dtype=np.uint32)

parametros = gerar_parametros_subcircuitos(z, s, n)
solver, x, d, y = construir_SMT(parametros, n)
z_estimado, falhas = estimar_z_com_falhas(solver, x, d)
falhas_max = maximizar_falhas_and(z, parametros, n, epsilon)

print("z verdadeiro:     ", z.tolist())
print("z' estimado:      ", z_estimado)
print("Falhas estimadas: ", falhas)
print("Falhas maximizadas:", falhas_max)

print("Número de falhas toleradas:", sum(falhas_max) if falhas_max else 0)

z verdadeiro:      [1, 1, 0, 1, 1, 0, 0, 0]
z' estimado:       [False, False, True, True, True, True, True, False]
Falhas estimadas:  [False, False, False, False, False, False, False, False]
Falhas maximizadas: [False, False, True, False, True, False, False, False]
Número de falhas toleradas: 2


In [52]:
n = 16
k = 48
epsilon = 0.6

z = np.random.randint(0, 2, size=n, dtype=np.uint8)
s = np.random.randint(0, 2**k, dtype=np.uint64)

parametros = gerar_parametros_subcircuitos(z, s, n)
solver, x, d, y = construir_SMT(parametros, n)
z_estimado, falhas = estimar_z_com_falhas(solver, x, d)
falhas_max = maximizar_falhas_and(z, parametros, n, epsilon)

print("z verdadeiro:     ", z.tolist())
print("z' estimado:      ", z_estimado)
print("Falhas estimadas: ", falhas)
print("Falhas maximizadas:", falhas_max)

print("Número de falhas toleradas:", sum(falhas_max) if falhas_max else 0)


z verdadeiro:      [0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0]
z' estimado:       [True, True, False, True, False, True, True, False, False, True, False, True, True, True, True, False]
Falhas estimadas:  [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
Falhas maximizadas: [False, False, False, True, True, True, True, False, False, True, False, False, True, True, True, True]
Número de falhas toleradas: 9


In [53]:
n = 32
k = 64
epsilon = 0.1 

z = np.random.randint(0, 2, size=n, dtype=np.uint8)
s = np.random.randint(0, 2**k, dtype=np.uint64)

parametros = gerar_parametros_subcircuitos(z, s, n)
solver, x, d, y = construir_SMT(parametros, n)
z_estimado, falhas = estimar_z_com_falhas(solver, x, d)
falhas_max = maximizar_falhas_and(z, parametros, n, epsilon)

print("z verdadeiro:     ", z.tolist())
print("z' estimado:      ", z_estimado)
print("Falhas estimadas: ", falhas)
print("Falhas maximizadas:", falhas_max)

print("Número de falhas toleradas:", sum(falhas_max) if falhas_max else 0)

z verdadeiro:      [1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0]
z' estimado:       [True, False, False, False, True, True, False, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, False, True, True, True, False, True, True, False]
Falhas estimadas:  [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
Falhas maximizadas: [False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False]
Número de falhas toleradas: 3
